In [1]:
import os
import sys
import math
import copy

if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
#from libra_py import *
from libra_py import units
from libra_py import QE_methods
from libra_py import scan


import py3Dmol   # molecular visualization
import matplotlib.pyplot as plt   # plots
%matplotlib inline 

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}

colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, s

## 1. CO molecule stretch

Generate 2 xyz files from the lines below

In general case, these files would be computed somehow.
Coordinates are in Angstrom

In [2]:
xyz1 = """2

C  0.0    0.0   0.0
O  2.0    0.0   0.0
"""

xyz2 = """2

C  0.0    0.0   0.0
O  4.0    0.0   0.0
"""

f = open("point1.xyz", "w")
f.write(xyz1)
f.close()

f = open("point2.xyz", "w")
f.write(xyz2)
f.close()

Read the remporary xyz files, then delete them

In [3]:
PT = {"C":12.0, "O":16.0 }
R1, E = QE_methods.read_md_data_xyz2("point1.xyz", PT)
os.system("rm point1.xyz")
print( E)
R1.show_matrix()
R2, E = QE_methods.read_md_data_xyz2("point2.xyz", PT)
os.system("rm point2.xyz")
print( E)
R2.show_matrix()

['C', 'O']
['C', 'O']


Create one path:

If the original points are mapped to 0 and 1 and there are 2 intervals, each interval is ds = 0.5

If the final mapping points are also 0 and 1, we have 3 points in total: 0.0, 0.5, 1.0

In [5]:
R, allxyz, s_axis = scan.make_path_xyz(R1, R2, E, s0=0.0, s1=1.0, npts=2, S0=0.0, S1=1.0)
#print allxyz
#R.show_matrix()

ndof =  6
nat =  2
ds =  0.5
Npts =  3


Now visualize the path

In [6]:
view = py3Dmol.view(width=800,height=400)  # linked=False,viewergrid=(3,2)\n
view.setBackgroundColor('0xeeeeee')                                     
view.zoomTo()                                                           
view.addModelsAsFrames(allxyz, 'xyz')
view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
view.animate({'reps':0, 'loop':'backandforward', 'step':1})
view.show()  

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

A path with more points. Also the final mapping points are selected -1 and 2, that means we'll have 7 points: -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0

In [7]:
s0, s1 = 0.0, 1.0
npts = 2
S0, S1 = -1.0, 2.0
R, allxyz, s_axis = scan.make_path_xyz(R1, R2, E, s0, s1, npts, S0, S1)

view = py3Dmol.view(width=800,height=400)  # linked=False,viewergrid=(3,2)\n
view.setBackgroundColor('0xeeeeee')                                     
view.zoomTo()                                                           
view.addModelsAsFrames(allxyz, 'xyz')
view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
view.animate({'reps':0, 'loop':'backandforward', 'step':1})
view.show()  

ndof =  6
nat =  2
ds =  0.5
Npts =  7


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 2. H2O molecule (or almost) bend 

In [8]:
r = 2.0 
theta = math.radians(30.0)
x1 = r * math.cos(theta)
x2 = -x1
y1 = r * math.sin(theta)
y2 = y1

xyz1 = """3

O  0.0    0.0   0.0
H  %5.3f  %5.3f   0.0
H  %5.3f  %5.3f   0.0
""" % (x1, y1, x2, y2)

theta = math.radians(60.0)
x1 = r * math.cos(theta)
x2 = -x1
y1 = r * math.sin(theta)
y2 = y1

xyz2 = """3

O  0.0    0.0   0.0
H  %5.3f  %5.3f   0.0
H  %5.3f  %5.3f   0.0
""" % (x1, y1, x2, y2)


f = open("point1.xyz", "w")
f.write(xyz1)
f.close()

f = open("point2.xyz", "w")
f.write(xyz2)
f.close()

In [10]:
PT = {"H":1.0, "O":16.0 }
R1, E = QE_methods.read_md_data_xyz2("point1.xyz", PT)
os.system("rm point1.xyz")
R2, E = QE_methods.read_md_data_xyz2("point2.xyz", PT)
os.system("rm point2.xyz")


s0, s1 = 0.0, 1.0
npts = 2
S0, S1 = 0.0, 1.0
R, allxyz, s_axis = scan.make_path_xyz(R1, R2, E, s0, s1, npts, S0, S1)


view = py3Dmol.view(width=800,height=400)  # linked=False,viewergrid=(3,2)\n
view.setBackgroundColor('0xeeeeee')                                     
view.zoomTo()                                                           
view.addModelsAsFrames(allxyz, 'xyz')
view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
view.animate({'reps':0, 'loop':'backandforward', 'step':1})
view.show()  

ndof =  9
nat =  3
ds =  0.5
Npts =  3


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol